## Problem 1 - The Spice Must Flow

House Atreides mines spice at a set of locations $M$ on Arrakis and ships it to a set of Houses $H$.  The cost per ton of producing spice at mine $m \in M$ is $p_m$.  The sand content (in percent) of the spice at mine $m \in M$ is $a_m$, the sulfur content of the spice at mine $m \in M$ is $s_m$, and the total tons of spice available at mine $m \in M$ is $u_m$.  Each House $h \in H$ requires $d_h$ tons of spice.  The cost (in Solari) of shipping a ton of spice from mine $m \in M$ to House $h \in H$ is $c_{mh}$.  It is required that the spice shipped to each House contain at most $\alpha$\% sand and at most $\gamma$\% sulfur in total.

Formulate a linear programming model that minimizes the cost of meeting demands of the Houses.  Be sure to clearly define the decision variables. (_Hint_: you will need variables representing the amount of spice shipped from each mine to each house.) Implement this general model in Julia and use it to solve the instance with data that is given along with
this homework assignment in the file "spice-data.ipnyb."

Using this LP:

Decision variable: x_mh represents tons of spice mined from mine m and sent to house h 

\begin{align*} 
\min sum(p_{m}*u_{m} + sum(e_{mh}*x_{mh}))\\       
   \mbox{s.t. } \phantom{+ 2x_2 } sum(x_{mh}) &\geq d_{h}\\
  \phantom{+ 2x_2} sum(a_m * x_{mh}) &\leq \alpha * d_h\\
  \phantom{+ 2x_2} sum(s_m * x_{mh}) &\leq \gamma * d_h\\
  x_{mh}\geq 0\\
\end{align*}  

In [1]:
##Spice-data

using Random

M = 20 # number of mining sites
H = 7 # number of houses

mines = 1:M # set of mining sites
houses = 1:H # set of houses

# set a seed for generating consistent data
seed = 494875
Random.seed!(seed)

# cost of spice production at each mine (in Solari)
mine_prod_cost =  [Random.rand(5:22) for i in mines]

# amount of spice at each mine (tons) 
mine_cap = [Random.rand(45:55) for i in mines]

# demand for spice (tons) at each House
demand = [Random.rand(50:80) for h in houses] 

# cost to transport between a (mine, house) pair
# rows are mines, columns are Houses
transport_cost = [round(2.5*Random.rand(),digits=2) 
    for i in mines, j in houses]

# percent of sand in spice at each mine
sand_per = [round(2.2*Random.rand()+5,digits=2) for i in mines]

# percent of sulfur in spice at each mine
sulf_per = [round(4*Random.rand()+3,digits=2) for i in mines]

# max percent of sand at each House
sand_lim = 6
# max percent of sulfur at each House
sulf_lim = 6
;

In [2]:
using JuMP, Clp

m = Model(Clp.Optimizer)

@variable(m, x[mines, houses] >= 0)

## objective: minimize total cost
@objective(m, Min, sum(mine_prod_cost[i] * mine_cap[i] + sum(transport_cost[i, j] * x[i, j] for j in houses) for i in mines)) ##minimize total cost

##mine cap constraint
for i in mines
    @constraint(m, sum(x[i, j] for j in houses) <= mine_cap[i])
end


for j in houses
    ##demand constraint
    @constraint(m, sum(x[i, j] for i in mines) >= demand[j]) 
    ##sand content constraint
    @constraint(m, sum(sand_per[i] * x[i, j] for i in mines) <= sand_lim * demand[j])
    ##sulfer content constraint
    @constraint(m, sum(sulf_per[i] * x[i, j] for i in mines) <= sulf_lim * demand[j])  
end

optimize!(m) 

println("objective value: ", objective_value(m))
println("spice shipments: ")
for i in mines
    for j in houses
        println("x[$i, $j] = ", round(value(x[i, j]), digits = 2))
    end
end


objective value: 16151.552793905921
spice shipments: 
x[1, 1] = 0.0
x[1, 2] = 0.0
x[1, 3] = 0.0
x[1, 4] = 0.0
x[1, 5] = 0.0
x[1, 6] = 0.0
x[1, 7] = 0.0
x[2, 1] = 0.0
x[2, 2] = 11.0
x[2, 3] = 0.0
x[2, 4] = 0.0
x[2, 5] = 0.0
x[2, 6] = 0.0
x[2, 7] = 0.0
x[3, 1] = 0.0
x[3, 2] = 0.0
x[3, 3] = 0.0
x[3, 4] = 0.0
x[3, 5] = 0.0
x[3, 6] = 18.66
x[3, 7] = 27.53
x[4, 1] = 0.0
x[4, 2] = 0.0
x[4, 3] = 0.0
x[4, 4] = 0.0
x[4, 5] = 0.0
x[4, 6] = 0.0
x[4, 7] = 0.0
x[5, 1] = 0.0
x[5, 2] = 48.0
x[5, 3] = 0.0
x[5, 4] = 0.0
x[5, 5] = 0.0
x[5, 6] = 0.0
x[5, 7] = 0.0
x[6, 1] = 0.0
x[6, 2] = 0.0
x[6, 3] = 0.0
x[6, 4] = 0.0
x[6, 5] = 2.2
x[6, 6] = 0.0
x[6, 7] = 0.0
x[7, 1] = 0.0
x[7, 2] = 0.0
x[7, 3] = 0.0
x[7, 4] = 0.0
x[7, 5] = 0.0
x[7, 6] = 0.0
x[7, 7] = 0.0
x[8, 1] = 0.0
x[8, 2] = 0.0
x[8, 3] = 6.55
x[8, 4] = 0.0
x[8, 5] = 42.45
x[8, 6] = 0.0
x[8, 7] = 0.0
x[9, 1] = 4.0
x[9, 2] = 0.0
x[9, 3] = 0.0
x[9, 4] = 0.0
x[9, 5] = 0.0
x[9, 6] = 0.0
x[9, 7] = 0.0
x[10, 1] = 0.0
x[10, 2] = 0.0
x[10, 3] = 1.45
x[10, 4] 

## Problem 2 - Spice Refineries

In the distant future, spice is one of the most valuable commodities that exists. However, it must be properly refined by House Atreides before leaving Arrakis to be sold across the galaxy. In order to refine the spice, House Atreides must operate its many refineries, which require water. In order to increase the output of spice at a refinery, there is a "cost" of 120 gallons of water per ton to operate the refinery at the higher output level. If instead the refinery  decreases production level of spice by 1 ton/day, the cost per ton is only 50 gallons of water. (E.g., if current production level is 10 tons and the refinery wants to increase output to 12 tons, the cost is 2 tons * 120 gallons/ton = 240 gallons.) 

Demand for spice in the next week is given in the following table:

|Day ($t$)|Demand (tons)|
|--:|--:|
|1 |40|
|2| 80|
|3|200|
|4|250|
|5|120|
|6|60|
|7|30|

The refinery is currently operating at an output level of 40 tons of spice per day. There are 10 tons of refined spice in inventory right now (day "0"). It costs 20 gallons of water per ton of spice to store up to 80 tons of spice in inventory. To operate a larger storage unit, additional tons (above 80) can be stored at a cost of 50 gallons of water per ton of spice. (Inventory is checked at the end of each day.) If the refinery is unable to meet demand for a given day, House Atreides will be penalized by the Empire and must pay 100 gallons of water per ton of spice short. 

Formulate and solve a linear program to determine the optimal production schedule to help House Atreides minimize the amount of water needed to operate the refineries. Give a mathematical formulation of the model as well as your Julia code and solution values.

In [3]:
using JuMP, Clp


demand = [40, 80, 200, 250, 120, 60, 30]

m = Model(Clp.Optimizer)

@variable(m, x[1:7] >= 0)  ## Surplus
@variable(m, y[1:7] in [0, 1])  ## production
@variable(m, z[1:7] >= 0)  ## 80+ surplus
@variable(m, w[1:7] >= 0)  ##80+ inventory
@variable(m, s[1:7] >= 0)  ## Spice in inventory
@variable(m, u[1:7] >= 0)  ## Penalty 


@objective(m, Min, sum(120 * y[t] + 50 * z[t] + 20 * w[t] + 100 * u[t] for t in 1:7))

##Production
@constraint(m, x[1] == 40)  
@constraint(m, x[t] == x[t-1] + y[t] + z[t] for t in 2:7)

##Inventory
@constraint(m, s[1] == 10 + x[1] - demand[1])  # Inventory at the end of day 1
@constraint(m, s[t] == s[t-1] + x[t] - demand[t] for t in 2:7)

##Surplus
@constraint(m, z[t] >= x[t] - 80)
@constraint(m, w[t] >= x[t] - y[t] - 80)
@constraint(m, u[t] >= demand[t] - x[t] - s[t] for t in 1:7)

@constraint(m, z[t] <= 80 * y[t])
@constraint(m, w[t] <= 80 * y[t])
@constraint(m, z[t] <= 80 * (1 - y[t]))
@constraint(m, w[t] <= 80 * y[t])

optimize!(m)


println("Total water used: ", objective_value(m), " gallons")
for t in 1:7
    println("Day ", t, ": Produce ", value(x[t]), " tons (additional ", value(y[t]), " tons)")
end
println("Inventory at the end of each day:")
for t in 1:7
    println("Day ", t, ": ", value(s[t]), " tons")
end


LoadError: MethodError: no method matching build_variable(::JuMP.var"#_error#99"{LineNumberNode}, ::Vector{ScalarVariable{Float64, Float64, Float64, Float64}}, ::Vector{Int64})

[0mClosest candidates are:
[0m  build_variable(::Function, [91m::VariableInfo[39m, ::Any...; kwargs...)
[0m[90m   @[39m [35mJuMP[39m [90m~/.julia/packages/JuMP/ptoff/src/[39m[90m[4mmacros.jl:1671[24m[39m
[0m  build_variable(::Function, ::AbstractArray{<:ScalarVariable}, [91m::MathOptInterface.AbstractScalarSet[39m)
[0m[90m   @[39m [35mJuMP[39m [90m~/.julia/packages/JuMP/ptoff/src/[39m[90m[4mmacros.jl:1820[24m[39m
[0m  build_variable(::Function, ::AbstractArray{<:ScalarVariable}, [91m::AbstractArray{<:MathOptInterface.AbstractScalarSet}[39m)
[0m[90m   @[39m [35mJuMP[39m [90m~/.julia/packages/JuMP/ptoff/src/[39m[90m[4mmacros.jl:1805[24m[39m
[0m  ...


## Problem 3 - Dice Delivery

You recently discovered you can make some extra money by leasing polyhedral dice to a group of friends playing Dungeons and Dragons. You recently lent 85 dice to 10 different people. Unforuntately, you made some mistakes and gave some people the wrong number of dice. The players have agreed to help with redistributing the dice, as long as they don't have to drive very far.  Suppose that the time to drive between each pair of people is 1.5 times the Euclidean distance to travel between each location given in the table below. The cost of gas is \\$0.40 per minute of driving. Although it is a bad strategy, also assume that the cost of traveling is per die (so transporting 2 dice a given distance costs 2$\times$1.5$\times$0.4$\times$distance).


|Player| 1|2|3|4|5|6|7|8|9|10|
|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|
|X coord|20 |18 |0 |35 |2 |11 |33 |2 |4 |12|
|Y coord|8| 13| 5 |7 |11| 15| 22| 7| 10|0|
|Dice requested|5 |7 |16 |4 |7 |15 |8 |10 |1 |12 |
|Current dice| 15| 6| 10| 1| 17| 9| 9| 3| 10| 5| 


How should the dice be redistributed to minimize total travel cost?

In [4]:
using JuMP, Clp, NamedArrays

players = 10
dice = 85

# Player coordinates and dice requests
xcoord = [20, 18, 0, 35, 2, 11, 33, 2, 4, 12]
ycoord = [8, 13, 5, 7, 11, 15, 22, 7, 10, 0]
requests = [5, 7, 16, 4, 7, 15, 8, 10, 1, 12]
current = [15, 6, 10, 1, 17, 9, 9, 3, 10, 5]

# Create a matrix to store the Euclidean distances between players and dice
distance = NamedArray(zeros(players, players), (1:players, 1:players))

for i in 1:players
    for j in 1:players
        distance[i, j] = sqrt((xcoord[i] - xcoord[j])^2 + (ycoord[i] - ycoord[j])^2)
    end
end


m = Model(Clp.Optimizer)

@variable(m, 0 <= x[1:players, 1:players] <= dice)


@objective(m, Min, sum(0.4 * 1.5 * distance[i, j] * x[i, j] for i=1:players, j=1:players))

@constraint(m, assignment[i in 1:players], sum(x[j, i] for j in 1:players) == requests[i])
@constraint(m, returns[i in 1:players], sum(x[i, j] for j in 1:players) == current[i])


optimize!(m)

println("Optimal dice redistribution:")
for i in 1:players
    for j in 1:players
        count = value(x[i, j])
        if count > 0
            println("Player ", i, " to Player ", j, " : ", count)
        end
    end
end
println("Objective Value: ", objective_value(m))


Optimal dice redistribution:
Player 1 to Player 1 : 5.0
Player 1 to Player 2 : 1.0
Player 1 to Player 4 : 2.0
Player 1 to Player 10 : 7.0
Player 2 to Player 2 : 6.0
Player 3 to Player 3 : 10.0
Player 4 to Player 4 : 1.0
Player 5 to Player 3 : 6.0
Player 5 to Player 5 : 7.0
Player 5 to Player 8 : 4.0
Player 6 to Player 6 : 9.0
Player 7 to Player 4 : 1.0
Player 7 to Player 7 : 8.0
Player 8 to Player 8 : 3.0
Player 9 to Player 6 : 6.0
Player 9 to Player 8 : 3.0
Player 9 to Player 9 : 1.0
Player 10 to Player 10 : 5.0
Objective Value: 147.69504021471812
Coin0506I Presolve 20 (0) rows, 100 (0) columns and 200 (0) elements
Clp0006I 0  Obj 0 Primal inf 170 (20)
Clp0006I 19  Obj 147.69504
Clp0000I Optimal - objective value 147.69504
Clp0032I Optimal objective 147.6950402 - 19 iterations time 0.002


## Problem 4 - Shortest path modeling

A company sells six types of boxes, ranging in volume from 17 to 35 cubic feet. The demand and size (volume) of each box are given in the table below.

|Box| 1|2|3|4|5|6|
|:--|:--|:--|:--|:--|:--|:--|
|Size| 35|29|28|25|19|17|
|Demand|40|35|55|70|15|40|

The variable cost per unit of producing each box is equal to its size divided by 10. In addition, if any positive number of a particular box is produced, a fixed cost of \\$10 is incurred.  For example, the variable cost to produce one type 1 box is \\$35 / 10 = 3.5, and so the total cost to produce 40 type 1 boxes would be \\$10 for the fixed cost, and 40*3.5=140 for variable costs, for a total cost of \\$150. If the company desires, demand for a box may be satisfied by a box of larger size.

Formulate the problem of finding the amount of each box to produce in order to minimize the total cost
to meet the customer demands as a shortest path problem. Solve the problem in Julia and report your solution.

In [5]:
using JuMP, Clp


size = [35, 29, 28, 25, 19, 17]
demand = [40, 35, 55, 70, 15, 40]

fix_cost = 10 * ones(6)
var_cost = size ./ 10
# incidence matrix 
# (rows are nodes, columns are arcs, entries represent whether arc enters (-1) or leaves (1) each node)
A = [150  272.5  465  710  762.5  902.5
      0   111.5  271  474  517.5  633.5
      0     0    164  261   402    514 
      0     0     0   185  222.5  322.5
      0     0     0    0    38.5  114.5
      0     0     0    0     0      78 ]


m = Model(Clp.Optimizer)


@variable(m, x[1:6, 1:6] >= 0)

@objective(m, Min, sum(x[i, j] * var_cost[i] + fix_cost[i] for i in 1:6, j in 1:6))

@constraint(m, d[j in 1:6], sum(x[i, j] for i in 1:6) == demand[j])


for i in 1:6
    for j in 1:6
        @constraint(m, sum(x[i, j] for i in 1:6) == A[j, i] * x[j, j])
    end
end

# Solve the problem
optimize!(m)

println("Total cost: \$", objective_value(m))

Total cost: $572.5
Coin0507I Presolve determined that the problem was infeasible with tolerance of 1e-08
Clp3003W Analysis indicates model infeasible or unbounded
Clp0006I 0  Obj 360 Primal inf 255 (6)
Clp0006I 2  Obj 572.5 Primal inf 476.6616 (16)
Clp0001I Primal infeasible - objective value 572.5
Clp0032I PrimalInfeasible objective 572.5 - 2 iterations time 0.002
